In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install med2image
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import glob
import re
import time
import sys

In [ ]:
def sort_list(l):
    convert = lambda text: int(text) if text.isdigit() else text
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(l, key = alphanum_key)

In [ ]:
def create_image(dirname, label):
    if dirname[-1] == "/":
        dirname = dirname[:-1]
    images = glob.glob(str(dirname) + "/*.png")

    # Sort images alphanumerically by filename to ensure that z loops from front to back
    images = sort_list(images)

    p = np.zeros((len(images), Image.open(images[0]).convert('L').size[0]))

    # setup toolbar
    toolbar_width = int(len(images)/5 + 1)
    sys.stdout.write(label + " [%s]" % (" " * toolbar_width))
    sys.stdout.flush()
    sys.stdout.write("\b" * (toolbar_width+1))

    # Loop through CT slices from front to back
    for z in range(len(images)):
        img = Image.open(images[z]).convert('L')  # convert image to 8-bit grayscale
        HEIGHT, WIDTH = img.size
        data = list(img.getdata()) # convert image data to a list of integers
        # convert that to 2D list (list of lists of integers)
        pixels = [data[offset:offset+WIDTH] for offset in range(0, WIDTH*HEIGHT, WIDTH)]

        # Loop from left to right on the CT slice
        for x in range(WIDTH):
            # Sum y values in the current x column
            sum = 0
            for y in range(HEIGHT):
                sum += pixels[y][x]
            # Assign sum to the point (x, z) on the coronal image - p[z][x] in the pixel array, 
            # since z represents height (rows) and x represents length (columns)
            p[len(images) - 1 - z][x] = sum

        if z % 5 == 0:
            # update the bar
            sys.stdout.write("-")
            sys.stdout.flush()
    sys.stdout.write("]\n")

    filename = 'img.jpg'

    array = np.array(p, dtype=np.uint8)
    xray = Image.fromarray(array)
    xray.save(filename)
    
    final_img = Image.open(filename)
    size = 300
    if final_img.size[1] < 300:
        final_img = final_img.resize((final_img.size[0], 300))    

    final_img.save(filename)

    ret = []
    ret.append(final_img)
    ret.append(len(images))

    return ret

In [ ]:
%cd /content/drive/My Drive/

In [ ]:
ret = []

thresh = 50

def floodfill(x, y):
    if matrix[x][y] > thresh: 
        matrix[x][y] = 0 
        ret.append((x,y))
        if x > 0:
            floodfill(x-1,y)
        if x < matrix.shape[0] - 1:
            floodfill(x+1,y)
        if y > 0:
            floodfill(x,y-1)
        if y < matrix.shape[1] - 1:
            floodfill(x,y+1)

def flood():
    arr = []
    for r in range(matrix.shape[0]):
        for c in range(matrix.shape[1]):
            if (matrix[r][c] > thresh):
                ret.clear()
                floodfill(r, c)
                m = []
                for i in ret:
                    m.append(i)
                arr.append(m)
    return arr

In [ ]:
from google.colab import files
import json
import random
import numpy as np
from scipy.spatial import ConvexHull
import matplotlib.pyplot as plt

sys.setrecursionlimit(300000)
data = {}

def show(index):
    ans = [] 
    a = arr[index]
    x = []
    y = []
    for tup in a:
      x.append(tup[0])
      y.append(tup[1])

    allPoints=np.column_stack((x,y))
    hull = ConvexHull(allPoints)

    plt.scatter(x, y, label= "o", color= "blue",  
            marker= "o", s=30) 

    for simplex in hull.simplices:
        ans.append((allPoints[simplex, 0][0], allPoints[simplex, 1][0]))
    
    return ans

# sizes = [12474, 6783, 6882, 6589, 6855, 6929, 6739, 7312, 6069, 6724, 6981, 6723, 6928, 6825, 6744, 6732, 6696, 6554, 6980, 6908, 6945, 6989, 6822, 6804, 6868, 6965, 6973, 7632, 6740, 6926, 6613, 12474, 11857, 13038, 11326, 12199, 10320, 12007, 13558, 11255, 11611, 6755]

for i in range(31,34):
    mask_name = './data/masks/Volume' + str(i)
    ret = create_image(mask_name, 'Creating mask ' + str(i) + ':')
    img = ret[0]
    l = ret[1]
    W, H = img.size
    img = img.resize((432, H))
    WIDTH, HEIGHT = img.size
    d = list(img.getdata()) 
    d = [d[offset:offset+WIDTH] for offset in range(0, WIDTH*HEIGHT, WIDTH)]
    matrix = np.asarray(d).astype(np.uint8)

    l = float(l)
    arr = flood()

    size = int(l * 21.4256 + 6031.48)
    fname = 'xray' + str(i) + '.jpg'
    name = fname + str(size)
    
    regions = []
    
    for idx in range(len(arr)):
        t = arr[idx]
        if (min(t)[0] == max(t)[0] or min(t, key = lambda q: q[1])[1] == max(t, key = lambda q: q[1])[1]):
            continue
        a = show(idx)
        all_points_x = []
        all_points_y = []
        for tup in a:
            all_points_x.append(int(tup[0]))
            all_points_y.append(int(tup[1]))
    
        shape_attributes = {}
        shape_attributes['name'] = 'polygon'
        shape_attributes['all_points_x'] = all_points_y
        shape_attributes['all_points_y'] = all_points_x

        image_quality = {}
        image_quality['good'] = True
        image_quality['frontal'] = True
        image_quality['good_illumination'] = True

        region_attributes = {}
        region_attributes['name'] = 'not_defined'
        region_attributes['type'] = 'unknown'
        region_attributes['image_quality'] = image_quality
        
        regions.append({
            'shape_attributes':shape_attributes,
            "region_attributes": region_attributes
        })

    file_attributes = {}
    file_attributes['caption'] = ''
    file_attributes['public_domain'] = 'no'
    file_attributes['image_url'] = ''

    data[name] = {}
    data[name]['filename'] = fname
    data[name]['size'] = size
    data[name]['regions'] = regions
    data[name]['file_attributes'] = file_attributes

with open('via_region_data.json', 'w') as outfile:
    json.dump(data, outfile)
files.download('via_region_data.json')